### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import plotly.graph_objects as go
# !pip install h3==3.7.3
import h3
import folium
import json
# !pip install geojson
import geojson
from folium.plugins import HeatMapWithTime
import datetime


# from google.colab import drive
# drive.mount('/content/drive/')

### Reading datasets

In [2]:
d_offer = pd.read_csv("datasets/data_offers.csv")
d_order = pd.read_csv("datasets/data_orders.csv")

# print the shape of offer dataset
print("The shape of offer dataset is: ", d_offer.shape)

# print the shape of order dataset
print("The shape of order dataset is: ", d_order.shape)


The shape of offer dataset is:  (334363, 2)
The shape of order dataset is:  (10716, 8)


In [3]:
d_order.head()

,order_datetime,origin_longitude,origin_latitude,m_order_eta,order_gk,order_status_key,is_driver_assigned_key,cancellations_time_in_seconds
0,18:08:07,-0.978916,51.456173,60.0,3000583041974,4,1,198.0
1,20:57:32,-0.950385,51.456843,NaN,3000583116437,4,0,128.0
2,12:07:50,-0.969520,51.455544,477.0,3000582891479,4,1,46.0
3,13:50:20,-1.054671,51.460544,658.0,3000582941169,4,1,62.0
4,21:24:45,-0.967605,51.458236,NaN,3000583140877,9,0,NaN


In [4]:
display(d_offer.sample(5, random_state=42))

,order_gk,offer_id
244971,3000627481054,300052086404
236783,3000631282042,300052342376
22714,3000624367715,300051909011
306983,3000557436646,300050769980
170256,3000588131807,300051201442


In [5]:
display(d_order.sample(5,random_state=42))

,order_datetime,origin_longitude,origin_latitude,m_order_eta,order_gk,order_status_key,is_driver_assigned_key,cancellations_time_in_seconds
10309,00:29:14,-0.966062,51.454821,NaN,3000555146103,4,0,44.0
6705,01:28:45,-0.970472,51.456725,NaN,3000595870644,4,0,121.0
304,08:22:36,-0.916708,51.439852,NaN,3000627783633,9,0,NaN
9143,07:32:27,-0.955580,51.453493,1078.0,3000622140805,4,1,1017.0
10593,04:39:55,-0.946358,51.439779,NaN,3000554645524,4,0,121.0


In [6]:
# change order_datetime into datetime format
d_order.order_datetime = pd.to_datetime(d_order.order_datetime)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8284\568184729.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d_order.order_datetime = pd.to_datetime(d_order.order_datetime)


In [7]:
print(f"Shape of dataset : {d_order.shape[0]}")

print(f"Number of unique orders : {d_order['order_gk'].nunique()}")

# rename m_order_eta as t_b4_order_arrival, cancellations_time_in_seconds as time_b4_cancellation
d_order.rename(columns={'cancellations_time_in_seconds':'time_b4_cancellation'}, inplace=True)

display(d_order.head())

Shape of dataset : 10716
Number of unique orders : 10716


,order_datetime,origin_longitude,origin_latitude,m_order_eta,order_gk,order_status_key,is_driver_assigned_key,time_b4_cancellation
0,2025-01-31 18:08:07,-0.978916,51.456173,60.0,3000583041974,4,1,198.0
1,2025-01-31 20:57:32,-0.950385,51.456843,NaN,3000583116437,4,0,128.0
2,2025-01-31 12:07:50,-0.969520,51.455544,477.0,3000582891479,4,1,46.0
3,2025-01-31 13:50:20,-1.054671,51.460544,658.0,3000582941169,4,1,62.0
4,2025-01-31 21:24:45,-0.967605,51.458236,NaN,3000583140877,9,0,NaN


#### Data Integrity Checking and Cleaning

In [8]:
print(d_order.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10716 entries, 0 to 10715
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   order_datetime          10716 non-null  datetime64[ns]
 1   origin_longitude        10716 non-null  float64       
 2   origin_latitude         10716 non-null  float64       
 3   m_order_eta             2814 non-null   float64       
 4   order_gk                10716 non-null  int64         
 5   order_status_key        10716 non-null  int64         
 6   is_driver_assigned_key  10716 non-null  int64         
 7   time_b4_cancellation    7307 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(3)
memory usage: 669.9 KB
None


In [9]:
print(d_offer.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334363 entries, 0 to 334362
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   order_gk  334363 non-null  int64
 1   offer_id  334363 non-null  int64
dtypes: int64(2)
memory usage: 5.1 MB
None


In [10]:
print(d_order.isnull().sum()/d_order.shape[0])

order_datetime            0.000000
origin_longitude          0.000000
origin_latitude           0.000000
m_order_eta               0.737402
order_gk                  0.000000
order_status_key          0.000000
is_driver_assigned_key    0.000000
time_b4_cancellation      0.318122
dtype: float64


In [11]:
print(d_offer.isnull().sum()/d_offer.shape[0])

order_gk    0.0
offer_id    0.0
dtype: float64


Order dataset contains some null value.

In [12]:
d_order[d_order.m_order_eta.isnull() | d_order.time_b4_cancellation.isnull()].head()

,order_datetime,origin_longitude,origin_latitude,m_order_eta,order_gk,order_status_key,is_driver_assigned_key,time_b4_cancellation
1,2025-01-31 20:57:32,-0.950385,51.456843,NaN,3000583116437,4,0,128.0
4,2025-01-31 21:24:45,-0.967605,51.458236,NaN,3000583140877,9,0,NaN
5,2025-01-31 21:21:23,-0.947011,51.456380,NaN,3000583117054,9,0,NaN
6,2025-01-31 07:58:15,-0.955637,51.470372,NaN,3000582791789,9,0,NaN
7,2025-01-31 07:53:46,-0.978230,51.454575,NaN,3000582791562,9,0,NaN


In [13]:
# common columns in both datasets
all_columns = pd.Series(list(d_offer)+list(d_order))
print(all_columns[all_columns.duplicated()])

6    order_gk
dtype: object


### Offers and Orders

In [14]:
print("The shape of order dataset is: ", d_order.shape)
print("The shape of offer dataset is: ", d_offer.shape)
df = pd.merge(d_order, d_offer, on='order_gk', how='inner', indicator=True)
print("The shape of merged dataset is: ", df.shape)

The shape of order dataset is:  (10716, 8)
The shape of offer dataset is:  (334363, 2)
The shape of merged dataset is:  (31268, 10)


In [15]:
df['order_status_key'] = np.where(df['order_status_key']==4,'Cancelled_by_client','Rejected_by_system')

In [16]:
cols = {'order_datetime':'order_time',
        'origin_longitude':'longitude',
        'origin_latitude':'latitude',
         'm_order_eta':'ETA',
         'order_gk':'order_id',
         'order_status_key':'order_status',
         'is_driver_assigned_key':'is_driver_assigned',
        'cancellations_time_in_seconds':'cancellation_time'}

df.rename(columns=cols,inplace=True)

In [17]:
display(df.sample(5, random_state=42))

,order_time,longitude,latitude,ETA,order_id,order_status,is_driver_assigned,time_b4_cancellation,offer_id,_merge
30194,2025-01-31 21:15:35,-0.969588,51.455780,299.0,3000555697076,Cancelled_by_client,1,223.0,300050725946,both
14748,2025-01-31 10:13:35,-0.955958,51.430362,NaN,3000626091676,Cancelled_by_client,0,91.0,300052014519,both
29083,2025-01-31 09:16:38,-1.033560,51.441967,NaN,3000594099225,Cancelled_by_client,0,68.0,300051458740,both
16926,2025-01-31 02:08:09,-0.967676,51.444489,NaN,3000626980467,Cancelled_by_client,0,125.0,300052031394,both
21530,2025-01-31 21:44:21,-0.972683,51.456762,NaN,3000554421440,Cancelled_by_client,0,11.0,300050671163,both


In [18]:
print(f"Number of cancelled orders after driver is assigned ....", \
      df[(df['is_driver_assigned'] ==1)].shape[0])

print(f"Number of cancelled orders before driver is assigned ....", \
      df[(df['is_driver_assigned'] == 0)].shape[0])


Number of cancelled orders after driver is assigned .... 8364
Number of cancelled orders before driver is assigned .... 22904


In [19]:
print(f"Number of orders cancelled by Customer : ",\
      df[(df['order_status'] == 'Cancelled_by_client')].shape[0])

print(f"Number of orders cancelledby System : ",\
      df[(df['order_status'] == 'Rejected_by_system')].shape[0])

Number of orders cancelled by Customer :  21795
Number of orders cancelledby System :  9473


In [20]:
print("Minimum time taken to cancel by a customer :" ,\
      df[(df['order_status'] == 'Cancelled_by_client')]['time_b4_cancellation'].min() , " seconds")
print("Maximum time taken to cancel by a customer :" ,\
      df[(df['order_status'] == 'Cancelled_by_client')]['time_b4_cancellation'].max() , " seconds")

print("Minimum time taken to cancel by System :" ,\
      df[(df['order_status'] == 'Rejected_by_system')]['time_b4_cancellation'].min(), " seconds")
print("Maximum time taken to cancel by System :" ,\
      df[(df['order_status'] == 'Rejected_by_system')]['time_b4_cancellation'].max(), " seconds")


Minimum time taken to cancel by a customer : 3.0  seconds
Maximum time taken to cancel by a customer : 4303.0  seconds
Minimum time taken to cancel by System : nan  seconds
Maximum time taken to cancel by System : nan  seconds


In [21]:
print("Average time taken to cancel by a customer :" ,\
      df[(df['order_status'] == 'Cancelled_by_client')]['time_b4_cancellation'].mean() , " seconds")
print("Minimum time taken to cancel by a system :" ,\
      df[(df['order_status'] == 'Rejected_by_system')]['time_b4_cancellation'].mean() , " seconds")

Average time taken to cancel by a customer : 163.86180316586373  seconds
Minimum time taken to cancel by a system : nan  seconds


In [22]:
df.sample( 5, random_state=42)

,order_time,longitude,latitude,ETA,order_id,order_status,is_driver_assigned,time_b4_cancellation,offer_id,_merge
30194,2025-01-31 21:15:35,-0.969588,51.455780,299.0,3000555697076,Cancelled_by_client,1,223.0,300050725946,both
14748,2025-01-31 10:13:35,-0.955958,51.430362,NaN,3000626091676,Cancelled_by_client,0,91.0,300052014519,both
29083,2025-01-31 09:16:38,-1.033560,51.441967,NaN,3000594099225,Cancelled_by_client,0,68.0,300051458740,both
16926,2025-01-31 02:08:09,-0.967676,51.444489,NaN,3000626980467,Cancelled_by_client,0,125.0,300052031394,both
21530,2025-01-31 21:44:21,-0.972683,51.456762,NaN,3000554421440,Cancelled_by_client,0,11.0,300050671163,both


### Question-1

Build up distribution of orders according to reasons for failure: cancellations before and after driver assignment, and reasons for order rejection. Analyse the resulting plot. Which category has the highest number of orders?

In [23]:
print("The shape of dataset is: ", df.shape)
orders_failure = df.groupby(['is_driver_assigned', 'order_status']).agg(
    order_count=('order_id', 'count'),
    mean_time_b4_cancellation=('time_b4_cancellation', 'mean'),
    max_time_b4_cancellation=('time_b4_cancellation', 'max')
).reset_index()
print("The shape of orders_failure is: ", orders_failure.shape)

The shape of dataset is:  (31268, 10)
The shape of orders_failure is:  (4, 5)


In [24]:
df[df.time_b4_cancellation < df.time_b4_cancellation.mean()].shape[0]

14816

In [25]:
df[(df.is_driver_assigned == 0) & (df.time_b4_cancellation < 121.771972) &
   (df.order_status == 'Cancelled_by_client')]['order_id'].nunique()

2267

In [26]:
df[(df.is_driver_assigned == 0) & (df.time_b4_cancellation < df.time_b4_cancellation.mean()) &
   (df.order_status == 'Cancelled_by_client')].head()

,order_time,longitude,latitude,ETA,order_id,order_status,is_driver_assigned,time_b4_cancellation,offer_id,_merge
1,2025-01-31 20:57:32,-0.950385,51.456843,NaN,3000583116437,Cancelled_by_client,0,128.0,300050986179,both
2,2025-01-31 20:57:32,-0.950385,51.456843,NaN,3000583116437,Cancelled_by_client,0,128.0,300050986174,both
3,2025-01-31 20:57:32,-0.950385,51.456843,NaN,3000583116437,Cancelled_by_client,0,128.0,300050986180,both
33,2025-01-31 06:33:52,-0.976216,51.433202,NaN,3000582765616,Cancelled_by_client,0,78.0,300050966073,both
42,2025-01-31 20:59:35,-0.972735,51.452489,NaN,3000583116472,Cancelled_by_client,0,45.0,300050986222,both


In [27]:
df[(df.is_driver_assigned == 0) & (df.time_b4_cancellation >= df.time_b4_cancellation.mean()) &
   (df.order_status == 'Cancelled_by_client')]['order_id'].nunique()

693

In [28]:
df[(df.is_driver_assigned == 1) & (df.order_status == 'Cancelled_by_client')]['order_id'].nunique()

2338

In [29]:
orders_failure.head()

,is_driver_assigned,order_status,order_count,mean_time_b4_cancellation,max_time_b4_cancellation
0,0,Cancelled_by_client,13435,120.313956,1179.0
1,0,Rejected_by_system,9469,NaN,NaN
2,1,Cancelled_by_client,8360,233.845694,4303.0
3,1,Rejected_by_system,4,NaN,NaN


In [30]:
import plotly.graph_objects as go
orders_failure_fig = go.Figure()

# Add bars for "Cancelled_by_client" with is_driver_assigned distinction
orders_failure_fig.add_trace(go.Bar(
    name='Cancelled_by_client',
    x=orders_failure[orders_failure['order_status'] == 'Cancelled_by_client']['is_driver_assigned'],
    y=orders_failure[orders_failure['order_status'] == 'Cancelled_by_client']['order_count'],
    text=orders_failure[orders_failure['order_status'] == 'Cancelled_by_client']['order_count'],
    textposition='auto'
))

# Add bars for "Rejected_by_system" with is_driver_assigned distinction
orders_failure_fig.add_trace(go.Bar(
    name='Rejected_by_system',
    x=orders_failure[orders_failure['order_status'] == 'Rejected_by_system']['is_driver_assigned'],
    y=orders_failure[orders_failure['order_status'] == 'Rejected_by_system']['order_count'],
    text=orders_failure[orders_failure['order_status'] == 'Rejected_by_system']['order_count'],
    textposition='auto'
))

# Add line for "mean_time_b4_cancellation" for "Cancelled_by_client" with data labels
orders_failure_fig.add_trace(go.Scatter(
    name='Mean Time Before Cancellation (Client)',
    x=orders_failure[orders_failure['order_status'] == 'Cancelled_by_client']['is_driver_assigned'],
    y=orders_failure[orders_failure['order_status'] == 'Cancelled_by_client']['mean_time_b4_cancellation'],
    mode='lines+markers+text',  # Add 'text' to display data labels
    line=dict(color='royalblue', width=2),
    marker=dict(size=8, symbol='circle'),
    text=orders_failure[orders_failure['order_status'] == 'Cancelled_by_client']['mean_time_b4_cancellation'].round(2) ,  # Show mean times as data labels
    textposition='top center',  # Positioning the text above the points
    yaxis='y2'  # Secondary y-axis for the line chart
))

# Update layout with secondary y-axis
orders_failure_fig.update_layout(
    barmode='group',
    xaxis_title='Driver Assigned (0 = No, 1 = Yes)',
    yaxis_title='Order Count',
    yaxis2=dict(
        title='Mean Time Before Cancellation (in seconds)',
        overlaying='y',
        side='right'
    ),
    title='Order Failures by Assignment and Status with Mean Time Before Cancellation',
    legend=dict(orientation="h", y=1.1, x=0.5, xanchor="center"),
    bargap = 0.7,
    bargroupgap = 0.05,
    width=900,  # Set the width of the chart
    height=500
)

orders_failure_fig.show()


In [31]:
import plotly.graph_objects as go

orders_failure_fig = go.Figure()

# Dynamically add bars for each unique order_status
for status in orders_failure['order_status'].unique():
    orders_failure_fig.add_trace(go.Bar(
        name=status,
        x=orders_failure[orders_failure['order_status'] == status]['is_driver_assigned'],
        y=orders_failure[orders_failure['order_status'] == status]['order_count'],
        text=orders_failure[orders_failure['order_status'] == status]['order_count'],
        textposition='auto'
    ))

# Add line for "mean_time_b4_cancellation" for "Cancelled_by_client" with rounded data labels
orders_failure_fig.add_trace(go.Scatter(
    name='Avg Time Before Cancellation (Client)',
    x=orders_failure[orders_failure['order_status'] == 'Cancelled_by_client']['is_driver_assigned'],
    y=orders_failure[orders_failure['order_status'] == 'Cancelled_by_client']['mean_time_b4_cancellation'],
    mode='lines+markers+text',
    line=dict(color='royalblue', width=2),
    marker=dict(size=8, symbol='circle'),
    text= orders_failure[orders_failure['order_status'] == 'Cancelled_by_client']['mean_time_b4_cancellation'].round(2) ,
    textposition='top center',
    yaxis='y2'
))

# Update layout with secondary y-axis and adjust chart size
orders_failure_fig.update_layout(
    barmode='group',
    xaxis_title='Driver Assigned (0 = No, 1 = Yes)',
    yaxis_title='Order Count',
    yaxis2=dict(
        title='Avg Time Before Cancellation in seconds',
        overlaying='y',
        side='right'
    ),
    title='Order Failures by Assignment and Status with Mean Time Before Cancellation',
    title_x = 0.5,  # Center the chart title
    width=800,  # Set the width of the chart
    height=500,  # Set the height of the chart
    legend=dict(orientation="h", y=1.1, x=0.5, xanchor="center")
)

orders_failure_fig.show()


It is observed that a high number of clients cancelled their order before a driver was assigned, implying that probably customers had waited too long to be assigned a driver and decided on other alternative transport means. There are `$$$$` order cancelled by the client, and `$$$$` rejected by the system. `$$$$` orders were cancelled by Clients before Driver was assigned while `$$$$` orders were rejected by system.After The Driver was assigned,`$$$$` orders were cancelled by the Client.

### Question-2

Plot the distribution of failed orders by hours. Is there a trend that certain hours have an abnormally high proportion of one category or another? What hours are the biggest fails? How can this be explained?

In [32]:
# Extract hour fro datetime column
df['order_hour'] = df.order_time.dt.hour

In [33]:
df[['order_hour','order_id','is_driver_assigned','order_status']].head()

,order_hour,order_id,is_driver_assigned,order_status
0,18,3000583041974,1,Cancelled_by_client
1,20,3000583116437,0,Cancelled_by_client
2,20,3000583116437,0,Cancelled_by_client
3,20,3000583116437,0,Cancelled_by_client
4,12,3000582891479,1,Cancelled_by_client


In [34]:
# Filter the DataFrame for 'Cancelled_by_client' and 'Rejected_by_system'
cancelled_df = df[df.order_status == 'Cancelled_by_client'].groupby('order_hour')['order_id'].count().reset_index()
rejected_df = df[df.order_status == 'Rejected_by_system'].groupby('order_hour')['order_id'].count().reset_index()

# Create the line chart
failed_orders_fig = go.Figure()

for status in ['Cancelled_by_client', 'Rejected_by_system']:
    failed_orders_fig.add_trace(go.Scatter(
        x=df[df.order_status == status].groupby('order_hour')['order_id'].count().index,
        y=df[df.order_status == status].groupby('order_hour')['order_id'].count(),
        mode='lines+markers',
        name=status,
        line = dict(width=2,dash='dot')

    ))


# Update layout
failed_orders_fig.update_layout(
    title='Count of Orders by Hour (Cancelled vs Rejected)',
    title_x=0.5,
    xaxis_title='Order Hour',
    yaxis_title='Order Count',
    width=800,
    height=600,
)

# Update x-axis to show every hour
failed_orders_fig.update_xaxes(
    tickvals=np.arange(0, 24, 1),  # Set the x-axis values to show every hour
    ticktext=[f"{hour}:00" for hour in np.arange(0, 24, 1)]  # Format the x-axis values as "hour:00"
)

# Show the figure
failed_orders_fig.show()


From the bar chart above , it is seen that the highest number of order fails occurred at `8:00`,followed by `21:00` and `23:00`

Failed orders are further investigated by hour by category, to obtain deeper insights into the cancellations by category.

### Question-3

Ploting the average time to cancellation with and without driver, by the hour to see it there are any outliers within the data and then proceed to remove them and draw further conclusion from the data

In [35]:
cancellation_time = df.groupby(['order_hour','is_driver_assigned'])\
                                    ['time_b4_cancellation'].mean().reset_index()

display(cancellation_time.head())

,order_hour,is_driver_assigned,time_b4_cancellation
0,0,0,115.126437
1,0,1,276.082822
2,1,0,100.593997
3,1,1,296.312081
4,2,0,121.305461


In [36]:
cancallation_time_fig = go.Figure()

for status in cancellation_time['is_driver_assigned'].unique():
    cancallation_time_fig.add_trace(go.Scatter(
        x=cancellation_time[cancellation_time['is_driver_assigned'] == status]['order_hour'],
        y=cancellation_time[cancellation_time['is_driver_assigned'] == status]['time_b4_cancellation'],
        mode='lines+markers',
        name=f'Driver Assigned: {status}',
        marker=dict(size=8),
        line=dict(width=2 , dash='dot')
    ))

# Update layout
cancallation_time_fig.update_layout(
    title='Average Cancellation Time by Hour and Driver Assignment',
    title_x=0.5,
    xaxis_title='Order Hour',
    yaxis_title='Time in Seconds',
    width=800,
    height=600,
)

# Update x-axis to show every hour
cancallation_time_fig.update_xaxes(
    tickvals=np.arange(0, 24, 1),  # Set the x-axis values to show every hour
    ticktext=[f"{hour}:00" for hour in np.arange(0, 24, 1)]  # Format the x-axis values as "hour:00"
)

# Show the figure
cancallation_time_fig.show()


### Question-4

Plot the distribution of average ETA by hours and explain any trend established.

In [37]:
df[['ETA','order_hour']].corr()

,ETA,order_hour
ETA,1.00000,-0.11881
order_hour,-0.11881,1.00000


Interpretation:
   - A value of -0.11881 indicates a weak negative correlation. This means that as `order_hour` increases (i.e., later in the day), there is a very slight tendency for the `ETA` to decrease, but the relationship is weak.
   - A correlation close to 0 (like in this case) suggests that the relationship between these two variables is not strong, meaning there isn't much linear dependence between `order_hour` and `ETA`.

In [38]:
average_eta = df.groupby('order_hour')['ETA'].mean().reset_index()

average_eta_fig = go.Figure()

average_eta_fig.add_trace(go.Scatter
                            (x=average_eta['order_hour'],
                             y=average_eta['ETA'],
                             mode='lines+markers',
                             marker=dict(size=8),
                             line=dict(width=2, dash='dot')
                            ))

average_eta_fig.update_layout(title='Average ETA by Hour',
                              title_x=0.5,
                              xaxis_title='Order Hour',
                              yaxis_title='ETA in Seconds',
                              width=800,
                              height=600)

average_eta_fig.update_xaxes(
    tickvals=np.arange(0, 24, 1),  # Set the x-axis values to show every hour
    ticktext=[f"{hour}:00" for hour in np.arange(0, 24, 1)]  # Format the x-axis values as "hour:00"
)

average_eta_fig.show()


The bar charts very closely matches the count of total failed orders per hour, indicating that the number of failed orders increases as the average waiting time of the client increases.

### Question-5

In [39]:
# Define a color scale based on the number of failed orders
def get_color(within_80_percent,fail_count):
    if within_80_percent == True:
        return 'Red'
    elif fail_count >= (order_count_by_hex['failed_order_count'].mean())/2:
        return 'purple'
    else:
        return 'Orange'


In [40]:
# Map each order to an h3 hexagon of resolution 8
df['hex_id'] = df.apply(lambda x: h3.geo_to_h3(x.latitude, x.longitude, 8), axis=1)

display(df[['order_id','longitude','latitude','order_status','is_driver_assigned','hex_id']].head())

,order_id,longitude,latitude,order_status,is_driver_assigned,hex_id
0,3000583041974,-0.978916,51.456173,Cancelled_by_client,1,88195d2b03fffff
1,3000583116437,-0.950385,51.456843,Cancelled_by_client,0,88195d2b19fffff
2,3000583116437,-0.950385,51.456843,Cancelled_by_client,0,88195d2b19fffff
3,3000583116437,-0.950385,51.456843,Cancelled_by_client,0,88195d2b19fffff
4,3000582891479,-0.969520,51.455544,Cancelled_by_client,1,88195d2b1dfffff


In [41]:
# Count the number of orders in each hexagon
order_count_by_hex = df['hex_id'].value_counts().reset_index()
order_count_by_hex.columns = ['hex_id', 'failed_order_count']
display(order_count_by_hex.head())

,hex_id,failed_order_count
0,88195d2b1dfffff,4488
1,88195d2b1bfffff,2603
2,88195d2b15fffff,2461
3,88195d2b11fffff,2191
4,88195d2b19fffff,1960


In [42]:
order_count_by_hex = order_count_by_hex.sort_values(by='failed_order_count', ascending=False)

display(order_count_by_hex.head())

,hex_id,failed_order_count
0,88195d2b1dfffff,4488
1,88195d2b1bfffff,2603
2,88195d2b15fffff,2461
3,88195d2b11fffff,2191
4,88195d2b19fffff,1960


In [43]:
order_count_by_hex['failed_order_count'].mean()

224.9496402877698

In [44]:
# Find hexagons that account for 80% of all orders
total_orders = order_count_by_hex['failed_order_count'].sum()
order_count_by_hex['cumulative_count'] = order_count_by_hex['failed_order_count'].cumsum()
eighty_percent_orders = total_orders * 0.8
order_count_by_hex['within_80_percent'] = order_count_by_hex['cumulative_count'] <= eighty_percent_orders



display(order_count_by_hex.head())

,hex_id,failed_order_count,cumulative_count,within_80_percent
0,88195d2b1dfffff,4488,4488,True
1,88195d2b1bfffff,2603,7091,True
2,88195d2b15fffff,2461,9552,True
3,88195d2b11fffff,2191,11743,True
4,88195d2b19fffff,1960,13703,True


In [45]:
# order_count_by_hex = order_count_by_hex.head(3)

# print(order_count_by_hex.shape)

In [46]:

# Visualize using Folium
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)

for index, row in order_count_by_hex.iterrows():
    hex_boundary = h3.h3_to_geo_boundary(row['hex_id'], geo_json=False)
    # hex_center = h3.h3_to_geo(row['hex_id'])

    # # Only if you want to see Markers
    # for loc in hex_boundary:
    #     folium.Marker(location=[loc[0], loc[1]], popup=f"({loc[0]}, {loc[1]})",
    #                    icon=folium.Icon(color='red', icon='pushpin')).add_to(m)


    popup_text = folium.Popup(f"Fail Orders: {int(row['failed_order_count'])}",
                        max_width=300)


    if hex_boundary:
        folium.PolyLine(locations=hex_boundary, color=get_color(row['within_80_percent'],row['failed_order_count']),
                        weight=1,
                        opacity=1,
                        fillOpacity=0.4,
                        fill=True,
                        popup=popup_text).add_to(m)

    else:
        print("Hex boundary is empty. Unable to plot the hexagon.")

display(m)


### Spatial Distribution of Orders: Where are the most orders originating from?

In [47]:
def h3index_into_geojson(row,col_name):
    """
    Transform hex_id into a geojson object.
    """
    geometry = {
    "type": "Polygon",
    "coordinates": [h3.h3_to_geo_boundary(h=row["hex_id"], geo_json=True)]
                }
    result = geojson.Feature(id=row["hex_id"], geometry=geometry, properties={"order_count":row[col_name]})
    return result

In [48]:
geojson_from_h3index = order_count_by_hex.apply(lambda x: h3index_into_geojson(x,'failed_order_count'), axis=1).values.tolist()
geojson_str: str = json.dumps(geojson.FeatureCollection(geojson_from_h3index))

max_order_count = order_count_by_hex['failed_order_count'].max()
min_order_count = order_count_by_hex['failed_order_count'].min()

colormap = matplotlib.colormaps.get_cmap("plasma")

In [49]:
# Initialize the map
intensity_map = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)


folium.GeoJson(
    geojson_str,
    style_function=lambda x: {
        'fillColor': matplotlib.colors.to_hex(colormap((x["properties"]["order_count"] - min_order_count) / (max_order_count - min_order_count))),
        'color': "black",
        'weight': 1,
        'fillOpacity': 0.7
    },
    popup=folium.GeoJsonPopup(fields=['order_count'], aliases=['Order Count:'], max_width=500)
).add_to(intensity_map)

# colorbar legend
color_min = matplotlib.colors.to_hex(colormap((min_order_count - min_order_count) / (max_order_count - min_order_count)))
color_max = matplotlib.colors.to_hex(colormap((max_order_count - min_order_count) / (max_order_count - min_order_count)))
color_bar = folium.LinearColormap([color_min, color_max], vmin=min_order_count, vmax=max_order_count)
color_bar.caption = "Order count"

intensity_map.add_child(color_bar)

# Display the map
display(intensity_map)


### Over Time

In [50]:
df.head()

,order_time,longitude,latitude,ETA,order_id,order_status,is_driver_assigned,time_b4_cancellation,offer_id,_merge,order_hour,hex_id
0,2025-01-31 18:08:07,-0.978916,51.456173,60.0,3000583041974,Cancelled_by_client,1,198.0,300050983403,both,18,88195d2b03fffff
1,2025-01-31 20:57:32,-0.950385,51.456843,NaN,3000583116437,Cancelled_by_client,0,128.0,300050986179,both,20,88195d2b19fffff
2,2025-01-31 20:57:32,-0.950385,51.456843,NaN,3000583116437,Cancelled_by_client,0,128.0,300050986174,both,20,88195d2b19fffff
3,2025-01-31 20:57:32,-0.950385,51.456843,NaN,3000583116437,Cancelled_by_client,0,128.0,300050986180,both,20,88195d2b19fffff
4,2025-01-31 12:07:50,-0.969520,51.455544,477.0,3000582891479,Cancelled_by_client,1,46.0,300050976275,both,12,88195d2b1dfffff


In [51]:
hour_order_fails = df.groupby(['order_hour','latitude', 'longitude'])['order_id'].count().reset_index()
hour_order_fails.columns = ['order_hour','latitude', 'longitude','failed_order_count']
display(hour_order_fails.head())

,order_hour,latitude,longitude,failed_order_count
0,0,51.403059,-0.975115,1
1,0,51.408597,-0.954554,1
2,0,51.413671,-0.959868,8
3,0,51.415417,-0.960242,5
4,0,51.416332,-0.959733,2


In [53]:
time_index = sorted(hour_order_fails['order_hour'].unique())
heat_data = []

for hour in time_index:
    data_for_hour = hour_order_fails[hour_order_fails['order_hour'] == hour][['latitude', 'longitude', 'failed_order_count']].values.tolist()
    heat_data.append(data_for_hour)


time_intensity_map = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)

HeatMapWithTime(heat_data, index=time_index, radius=10, auto_play=True, max_opacity=0.7).add_to(time_intensity_map)

# time_intensity_map.save("/content/drive/MyDrive/Colab Notebooks/Insights from Failed Orders/datasets/time_intensity_map.html")

display(time_intensity_map)
